# Six-bar mechanism balancing

In [1]:
from differential_evolution import differential_evolution
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from BetaShF import *
from BetaShM import *
import scipy as sp
import torch
import time

assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0


c:\Users\lalor\miniconda3\envs\pyTorch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Util

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device('cpu')
print("Device", device)

Device cpu


In [3]:
seed = 1
torch.manual_seed(seed)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = torch.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [5]:
'''
def saveData(samples, fitness, forces, moments):
    now = str(int(time.time()))
    np.savetxt(now + "Population.txt", samples)
    np.savetxt(now + "Fitness.txt", fitness)
    np.savetxt(now + "ShForces.txt", forces)
    np.savetxt(now + "ShMoments.txt", moments)
    return now
'''

'\ndef saveData(samples, fitness, forces, moments):\n    now = str(int(time.time()))\n    np.savetxt(now + "Population.txt", samples)\n    np.savetxt(now + "Fitness.txt", fitness)\n    np.savetxt(now + "ShForces.txt", forces)\n    np.savetxt(now + "ShMoments.txt", moments)\n    return now\n'

In [6]:
def logSample(now, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open(name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(now + "Population.txt", s)
    appendToFile(now + "Fitness.txt", str(fitness))
    appendToFile(now + "ShForces.txt", str(force))
    appendToFile(now + "ShMoments.txt", str(moment))

## Problem definition

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

$$
f(x) = \alpha ShF(x) + (1 - \alpha) ShM(x)
$$
$$
\therefore \alpha = \frac{f(x) - ShM(x)}{ShF(x) - ShM(x)}
$$

In [7]:
def objective(s, ShF, ShM, alpha):
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

## Sample generation

In [8]:
#nSamples, nGen = 300, 500 # Tune here
nSamples, nGen = 1, 1

nVariables = 15
nWeights = nVariables // 3
samples = torch.zeros((nSamples, nVariables))
bounds = torch.zeros((nVariables, 2))
fitness = torch.zeros((nSamples))
shForces = torch.zeros((nSamples))
shMoments = torch.zeros((nSamples))

### Differential Evolution

#### Define boundaries

In [9]:
for v in range(nVariables):
    bounds[v] = torch.as_tensor([0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16])
bounds = bounds.to(device)
print(bounds)

tensor([[-0.1600,  0.1600],
        [-0.1600,  0.1600],
        [ 0.0050,  0.0400],
        [-0.1600,  0.1600],
        [-0.1600,  0.1600],
        [ 0.0050,  0.0400],
        [-0.1600,  0.1600],
        [-0.1600,  0.1600],
        [ 0.0050,  0.0400],
        [-0.1600,  0.1600],
        [-0.1600,  0.1600],
        [ 0.0050,  0.0400],
        [-0.1600,  0.1600],
        [-0.1600,  0.1600],
        [ 0.0050,  0.0400]])


In [10]:
eTimeDE, goodSols = 0, 0
startTime = str(int(time.time()))
print("Saving data with the prefix:", startTime)
for s in range(nSamples):
    alpha = torch.normal(0.5, torch.tensor([0.20])) # To get values near 0.5
    alpha = torch.min(torch.tensor([1, torch.max(torch.tensor([0, alpha]))])) # Box constraints [0, 1]
    start = time.perf_counter()
    #r = differential_evolution(objective, bounds, popSize = 50, nMax = nGen, args = (ShF, ShM, alpha), seed = seed, device = device)
    r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM, alpha), maxiter = nGen, popsize = 50, polish = False)
    end = time.perf_counter()
    #sample, fitness, force, moment = r['x'], r["fun"], ShF(r['x']), ShM(r['x'])
    sample, fitness, force, moment = r.x, r.fun, ShF(r.x), ShM(r.x)
    if force < 1 and moment < 1:
        goodSols += 1
        logSample(startTime, sample, fitness, force, moment)
    print("Sample ", s, ":\tshF: ", force, "\tshM: ", moment, "\talpha = ", alpha, sep = "")
    eTimeDE += (end - start) #Time in seconds
eTimeDE /= nSamples

Saving data with the prefix: 1676262509
Sample 0:	shF: 2.4313841778081606	shM: 3.51655887191901	alpha = tensor(0.6323)


In [11]:
print("Average time of execution:", eTimeDE, "seconds. It was run", nSamples, "times.")

Average time of execution: 17.7131062 seconds. It was run 1 times.


In [12]:
print("Went from ", nSamples, " samples to ", goodSols, " after cleaning data", sep = "")

Went from 1 samples to 0 after cleaning data


### Pymoo

In [13]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

ModuleNotFoundError: No module named 'pymoo'

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=nVariables,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=bounds[:, 0],
                         xu=bounds[:, 1]
        )

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = ShF(x)
        f2 = ShM(x)

        out["F"] = [f1, f2]
        out["G"] = []


problem = MyProblem()

In [ ]:
algorithm = NSGA2(
    pop_size=nSamples, 
    #n_offsprings=nSamples, # Default is none and sets n_offsprings = pop_size
    sampling=FloatRandomSampling(),
    crossover=UniformCrossover(prob=0.75),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
termination = get_termination("n_gen", nGen)

In [ ]:
start = time.perf_counter()
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
end = time.perf_counter() 
eTimeMOO = (end - start)

X = res.X
F = res.F

In [ ]:
print("Time of execution:", eTimeMOO, "seconds.")

In [ ]:
print(X.shape)
print(F.shape)
samples = np.copy(X)
fitness = np.average(F, axis = 1)
shForces = F[:, 0]
shMoments = F[:, 1]

In [ ]:
samples, fitness, shForces, shMoments = cleanData(samples, fitness, shForces, shMoments)
print("Went from ", nSamples, " samples to ", samples.shape[0], " after cleaning data", sep = "")

In [ ]:
prefix = saveData(samples, fitness, shForces, shMoments)
print("Saving data with the prefix:", prefix)